# import

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import math
import glob
import re
import random
import collections
import os
import sys
import pickle

# data

In [2]:
botname = 'แว่นเป๋อ❄️'
in_folder = '/home/aroundy/Documents/Chula/3/NLP/stanford-tensorflow-tutorials/assignments/chatbot/Wannta/Linechat/'
out_folder = '/'.join(in_folder.split('/')[:-2])+'/'

In [77]:
import os
import code
convs = []
for file in os.listdir(in_folder):
    if(file in ["out.txt","Line chat.zip"]):
        continue
    with open(in_folder+file, 'r') as f:
        print(file)

        conv = []
        isFirst = True
        isMultipleLine = False
        is_human_first = True

        combined_sent = []
        prev_send = ''
        
        for line in f:            
            sender = ''
            sent = ''
            spl = line.strip().split('\t')
            
            if(len(spl) == 3):
                sender = spl[1]
                sent = spl[2]
                

                if(sent in ['[Photo]','[Sticker]','[File]' , '[Video]']):
                    continue
                if(sent[0] == '☎'):
                    continue
                
                if(sent[0] == '"'):
                    sent = sent[1:]
                    isMultipleLine = True
                    
                if(sender != prev_send):
                    conv.append(prev_send + "\t" +' '.join(combined_sent))
                    combined_sent = [sent]
                    prev_send = sender
                    
                else:
                    combined_sent.append(sent)
            
            elif(isMultipleLine):
                if(len(line) > 1 and line[-2] == '"'):
                    isMultipleLine = False
                combined_sent.append(line.strip().strip('"'))
                

#         combined_sent.append(sent)
        if(prev_send == botname):
            conv.append(prev_send + '\t' +' '.join(combined_sent))
        
        cidx=1
        while(conv[cidx].split('\t')[0] == botname):
            cidx+=1
        
        convs.append(conv[cidx:])
        
with open(out_folder+'debug-sentences-1.txt', 'w') as o:
    for con in convs:
        for sent in con:
            o.write(sent + '\n')
            
with open(out_folder+'sentences-1.txt', 'w') as o:
    for con in convs:
        for sent in con:
            o.write(sent.split('\t')[1] + '\n')            

[LINE] Chat with nnn..txt
[LINE] Chat with Minato Namikaze.txt
[LINE] Chat with воѕѕ.txt
[LINE] Chat with Jaja _).txt
[LINE] Chat with evesuju☆이프슈주.txt
[LINE] Chat with findingfon.txt
[LINE] Chat with waii..txt
[LINE] Chat with Intaniger.txt
[LINE] Chat with Jame.txt
[LINE] Chat with Tong.txt
[LINE] Chat with Fair 1.txt
[LINE] Chat with Ter.txt
[LINE] Chat with OaT.txt
[LINE] Chat with num.txt
[LINE] Chat with Nice.txt
[LINE] Chat with Tam♡.txt
[LINE] Chat with K.txt
[LINE] Chat with .เจี๊ยบ..txt
[LINE] Chat with Tan.txt
[LINE] Chat with Thipok123.txt
[LINE] Chat with momo.txt
[LINE] Chat with -Gun.txt
[LINE] Chat with Brighttiies 明.txt
[LINE] Chat with .ŇʋИә ♡.txt
[LINE] Chat with TheBrave Yongyee.txt
[LINE] Chat with Progorn.txt
[LINE] Chat with papoy.txt
[LINE] Chat with Fair.txt
[LINE] Chat with cp..txt
[LINE] Chat with Proud'.txt
[LINE] Chat with Warm.txt
[LINE] Chat with petchphi.txt
[LINE] Chat with Napatt.txt
[LINE] Chat with pibi.txt
[LINE] Chat with Noey.txt
[LINE] Chat 

In [ ]:
conver = []
with open(out_path+'sentences.txt') as f:
    for line in f:
        conver.append(line)

# tokenize
- This tokenize used Deepcut.
- May use TokenizeWords.ipynb which used Pythainlp instead. 
  If so, ignore this topic and run that file.

In [5]:
import deepcut
deepcut.tokenize("สวัสดีเพื่อนๆ")

Using TensorFlow backend.
/home/aroundy/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


['สวัสดี', 'เพื่อน', 'ๆ']

In [ ]:
%%time
token = []
for sent in conver:
    token.append(deepcut.tokenize(sent.strip()))
print(token[:5])

In [ ]:
token_set = set()
for words in token:
    token_set = token_set.union(words)

token_list = list(token_set)
print(token_list)

In [78]:
#fromm Tipok code style
token = []
with open('tokenized_out.txt','r') as f:
    for line in f:
        token.append(line.split())

# Dictionary
- If you want sentense to have a start/stop word, Comment out thw two line before return

In [79]:
def create_dataset_dict(input_list):
    words = []
    for review in input_list:
        for word in review:
            words.append(word)
    word_count = list()
    #use set and len to get the number of unique words
    word_count.extend(collections.Counter(words).most_common(len(set(words))))
    
    #threshold = len(input_text)//1e6
    threshold = 1
    print("Threshold", threshold)
    
    word_unk = set()
    num_unk = 0
    idx = len(word_count)-1
    while word_count[idx][1] <= threshold:
        word_unk.add(word_count[idx][0])
        num_unk += word_count[idx][1]
        idx -= 1

    word_count = word_count[:idx+1]
    
    #include a token for unknown word
    word_count.append(("เหรอ",num_unk))
    word_count = sorted(word_count, key=lambda x: -x[1])
    #word_count = [x for x in word_count if x[0] not in word_unk]
    
    #print out 10 most frequent words
    print(word_count[:10])
    print("#unk" ,num_unk)
    
    dictionary = dict()
    dictionary["for_keras_zero_padding"] = 0
    for word in word_count:
        dictionary[word[0]] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    datas = list()
    for review in input_list:
        data = []
        for word in review:
            if word in word_unk:
                data.append(dictionary["เหรอ"])
            else:
                data.append(dictionary[word])
        datas.append(data)
#     dictionary["<s>"] = len(dictionary)
#     dictionary["</s>"] = len(dictionary)

    return datas,dictionary, reverse_dictionary

In [80]:
dataset,dictionary, reverse_dictionary=create_dataset_dict(token)

Threshold 1
[('555', 13511), ('ไป', 7497), ('ไม่เข้าใจอ่ะ พูดอย่างอื่นได้ป่าว', 7490), ('เค้า', 5302), ('อ่ะ', 4971), ('แก', 4600), ('ไม่', 4438), ('ก็', 4437), ('มัน', 4099), ('จะ', 4072)]
#unk 7490


In [81]:
print("len data set:", len(dataset))
print("len dictionary:", len(dictionary))
print("output sample (dataset):",dataset[:10])
print("output sample (dictionary):",{k: dictionary[k] for k in list(dictionary)[:10]}, "\ndictionary size: ",len(dictionary))
print("output sample (reverse dictionary):",{k: reverse_dictionary[k] for k in list(reverse_dictionary)[:10]})

len data set: 35434
len dictionary: 8603
output sample (dataset): [[105, 85, 43, 269], [867, 1276, 104, 1260], [307, 36, 7, 47, 112, 81], [6, 178, 112, 207, 1409, 930, 905, 173, 31, 1, 1469, 100, 35, 970, 590, 64, 6, 2, 142, 218, 57, 173, 181, 23, 2, 919, 23, 1155, 17, 1], [2452, 21, 29, 112, 662, 2, 2216, 17, 178, 1059, 905, 5131, 1940, 540, 3523], [112, 82, 433, 199, 7, 2996, 42, 9, 358, 8, 191, 152], [2606, 1803, 434, 1, 102, 1029, 204, 83, 57, 2, 2216, 17, 34, 1029, 6295, 516, 770], [8, 2, 2216, 12, 1640, 199, 375, 202, 7, 43, 1803, 8, 561, 310], [722, 250, 545, 28, 1015, 752, 2217, 27], [43, 498, 141, 499, 291, 655, 1]]
output sample (dictionary): {'for_keras_zero_padding': 0, '555': 1, 'ไป': 2, 'ไม่เข้าใจอ่ะ พูดอย่างอื่นได้ป่าว': 3, 'เค้า': 4, 'อ่ะ': 5, 'แก': 6, 'ไม่': 7, 'ก็': 8, 'มัน': 9} 
dictionary size:  8603
output sample (reverse dictionary): {0: 'for_keras_zero_padding', 1: '555', 2: 'ไป', 3: 'ไม่เข้าใจอ่ะ พูดอย่างอื่นได้ป่าว', 4: 'เค้า', 5: 'อ่ะ', 6: 'แก', 7: 'ไม่', 8: '

In [82]:
out_path = '/home/aroundy/Documents/Chula/3/NLP/stanford-tensorflow-tutorials/assignments/chatbot/Wannta/prepro/'

- run this one if you want a start/stop word.

In [48]:
human = []
bot = []
even = True
for line in dataset:
    if (even):
        human.append(line)
        even = False
    else:
        bot.append(line)
        even = True
test_train_split = 0.8
        
human_train = human[:int(len(human)*test_train_split)]
human_test = human[int(len(human)*test_train_split):]
bot_train = bot[:int(len(bot)*test_train_split)]
bot_test = bot[int(len(bot)*test_train_split):]

start = str(dictionary["<s>"])
end = str(dictionary["</s>"])

with open(out_path+'human_train.txt', 'w') as o:
    for line in human_train:
        line = " ".join([str(x) for x in line])
        o.write(line+'\n')
with open(out_path+'bot_train.txt', 'w') as o:
    for line in bot_train:
        line = " ".join([start]+[str(x) for x in line]+[end])
        o.write(line+'\n')
with open(out_path+'human_test.txt', 'w') as o:
    for line in human_test:
        line = " ".join([str(x) for x in line])
        o.write(line+'\n')
with open(out_path+'bot_test.txt', 'w') as o:
    for line in bot_test:
        line = " ".join([start]+[str(x) for x in line]+[end])
        o.write(line+'\n')

In [49]:
dict_list = []
with open(out_path+'vocab.txt','w') as o:
    for value in dictionary:
        o.write(value+'\n')

- Otherwise, run this.

In [83]:
human = []
bot = []
even = True
for line in dataset:
    if (even):
        human.append(line)
        even = False
    else:
        bot.append(line)
        even = True
test_train_split = 0.8
        
human_train = human[:int(len(human)*test_train_split)]
human_test = human[int(len(human)*test_train_split):]
bot_train = bot[:int(len(bot)*test_train_split)]
bot_test = bot[int(len(bot)*test_train_split):]

# start = str(dictionary["<s>"])
# end = str(dictionary["</s>"])

with open(out_path+'human_train.txt', 'w') as o:
    for line in human_train:
        line = " ".join([str(x) for x in line])
        o.write(line+'\n')
with open(out_path+'bot_train_nostart.txt', 'w') as o:
    for line in bot_train:
        line = " ".join([str(x) for x in line])
        o.write(line+'\n')
with open(out_path+'human_test.txt', 'w') as o:
    for line in human_test:
        line = " ".join([str(x) for x in line])
        o.write(line+'\n')
with open(out_path+'bot_test_nostart.txt', 'w') as o:
    for line in bot_test:
        line = " ".join([str(x) for x in line])
        o.write(line+'\n')

In [84]:
dict_list = []
with open(out_path+'vocab_nostart.txt','w') as o:
    for value in dictionary:
        o.write(value+'\n')

In [85]:
dictionary["UNK"]

3515

ready for training in chat.py 